## Demo Notebook: How to evaluate your predicted scenes

In this notebook, we will demonstrate all the steps you need to evaluate and visualize the results of this pipeline. For every scene you need to make sure you have the following:
- A predicted scene result in `results/semantic_labels` as `{pairing}_{scene}_{masks}.pickle`
- A GT Point Cloud / mesh of the Scene to project colors and predicted semantics back to the original format in `data/{scene}` as `{scene}_vh_clean_2.ply`
- A GT Text file indicating the correct semantics for every point of the point cloud / vertex of the mesh in `data/scannet_gt_scenes/train` as `{scene}.txt`

### Real-World Data: ScanNet v2

This demo is done for ScanNet, but other datasets will follow the same steps

To evaluate the predicted results, we need to packproject them into the format of the given GT. This in combination with the visualization is done with the NERFSTUDIO Viewer. To bring the data into the format and compare it to the Baseline, we first run the baselines training script

Make sure to run this cell in the opennerf environment


In [ ]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

import files
from eval.evaluator import Evaluator

dataset = 'scannet'
scene = 'scene0062_00'
masks = 'ovseg'
pairing = 'assignment'

evaluator = Evaluator(dataset=dataset, scene=scene, masks=masks, pairing=pairing)

In [ ]:
evaluator.run_opennerf()

After this, we can run the function to evaluate the predicted semantics. For some reason this does not behave well in a Notebook Cell, even though it runns completely normal when called in scripts. We provide the way to call it in the pipeline as well as the execution in the shell (its the same command the method calls) to visualize how it will look.

In [ ]:
#In your skript, just call it like this:
#evaluator.eval_scene()

#Note: Since for some reason this does not run in a notebook cell but otherwise works in the command line as it should, this cell is run as a shell command (same command the method calls)
EXPERIMENT_NAME = 'benchmark'
!{files.OPENNERF_ENV_PYTHON} {files.BASE_PATH}src/eval/{dataset}_semantics.py interpolate --interpolation-steps=1 --pose_source=train --load-config={files.OPENNERF_PATH}outputs/{dataset}_{scene}/opennerf/{EXPERIMENT_NAME}/config.yml --colormap-options.colormap=pca --output_path={files.OPENNERF_PATH}outputs/{dataset}_{scene}/opennerf/{EXPERIMENT_NAME}/ --rendered-output-names=rgb --eval-num-rays-per-chunk=500 --downscale-factor=1 --semantics_input_path={files.RESULT_PATH}semantic_labels/{pairing}_{scene}_{masks}.pickle --semantics_output_path={files.RESULT_PATH}pr_semantics/semantics_{pairing}_{scene}_{masks}.txt

If you want to visualize the results, just follow the steps in the output above and the 3D Viewer in the Browser is activated

After creating the corresponding `prediction.txt` file we can evaluate them with respect to the `gt.txt` files, and you see your predicted results

In [ ]:
evaluator.eval_semantics()